##LeNet
![alt text](https://i.gyazo.com/57d81430a6a171f74ff6aeb8afa0ab4f.png)

In [ ]:
import torch
import random
import numpy as np

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True

In [ ]:
import torchvision.datasets

In [ ]:
MNIST_train = torchvision.datasets.MNIST('./', download=True, train=True)
MNIST_test = torchvision.datasets.MNIST('./', download=True, train=False)


In [ ]:
X_train = MNIST_train.train_data
y_train = MNIST_train.train_labels
X_test = MNIST_test.test_data
y_test = MNIST_test.test_labels

In [ ]:
len(y_train), len(y_test)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(X_train[0, :, :])
plt.show()
print(y_train[0])

In [ ]:
'''y
Первое отличие заключается в том, что, в отличие от полносвязанной сети, 
которая видела картинку как один вектор длинный, мы хотим в конволюционную (сверточную)
сеть передавать картинку как трёхмерный тензор. 
Первый канал -- это глубина картинки, в черно-белой картинке это 1 канал с яркостью серого пикселя. 
А в RGB картинке будут RGB каналы. Соответственно, 
мы должны нашу картинку, которая на вход пришла (она просто "28 на 28"), разжать до "1 на 28 на 28". 
'''

# unsqueeze: 
# Returns a new tensor with a dimension of size one inserted at the specified position.
# The returned tensor shares the same underlying data with this tensor.

X_train = X_train.unsqueeze(1).float()
X_test = X_test.unsqueeze(1).float()


In [ ]:
X_train.shape

In [ ]:
class LeNet5(torch.nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__() 
        
        self.conv1 = torch.nn.Conv2d(
            in_channels=1, out_channels=6, kernel_size=5, padding=2) # 28x28 ->(pad) 32x32 -> 28x28
        self.act1  = torch.nn.ReLU()
        self.pool1 = torch.nn.AvgPool2d(kernel_size=2, stride=2) # 28x28 -> 14x14
       
        self.conv2 = torch.nn.Conv2d(
            in_channels=6, out_channels=16, kernel_size=5, padding=0) # 14x14 -> 10x10
        self.act2  = torch.nn.ReLU()
        self.pool2 = torch.nn.AvgPool2d(kernel_size=2, stride=2) # 10x10 -> 5x5
        
        self.fc1   = torch.nn.Linear(5 * 5 * 16, 120) # 400 -> 120
        self.act3  = torch.nn.ReLU()
        
        self.fc2   = torch.nn.Linear(120, 84)
        self.act4  = torch.nn.ReLU()
        
        self.fc3   = torch.nn.Linear(84, 10)
    
    def forward(self, x):
        # Входной тензор X -- это, батч из картинок
        x = self.conv1(x)
        x = self.act1(x)
        x = self.pool1(x)
        
        x = self.conv2(x)
        x = self.act2(x)
        x = self.pool2(x)
        
        # У PyTorch-тензоров есть функция view, которая наш тензор преобразует к нужной размерности. 
        # Первая размерность будет x.size[0] -- это размер батча, а дальше тензор будет одномерный,
        # соответственно мы вот эти три размерности должны просто перемножить и получить 400.
        x = x.view(x.size(0), x.size(1) * x.size(2) * x.size(3))

        x = self.fc1(x)
        x = self.act3(x)
        x = self.fc2(x)
        x = self.act4(x)
        x = self.fc3(x)
        
        return x
    
lenet5 = LeNet5()

In [ ]:
# чтобы работать на GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
lenet5 = lenet5.to(device)

In [ ]:
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(lenet5.parameters(), lr=1.0e-3)

In [ ]:
# описание процесса обучения

batch_size = 50

test_accuracy_history = []
test_loss_history = []

# X_test = X_test.to(device)
# y_test = y_test.to(device)

for epoch in range(20):
    order = np.random.permutation(len(X_train))
    for start_index in range(0, len(X_train), batch_size):
        optimizer.zero_grad()
        
        batch_indexes = order[start_index:start_index+batch_size]
        
        X_batch = X_train[batch_indexes].to(device)
        y_batch = y_train[batch_indexes].to(device)
        
        preds = lenet5.forward(X_batch) 
        
        loss_value = loss(preds, y_batch)
        loss_value.backward()
        
        optimizer.step()
        

        
#     test_preds = lenet5.forward(X_test)
#     test_loss_history.append(loss(test_preds, y_test).data.cpu())
#     accuracy = (test_preds.argmax(dim=1) == y_test).float().mean().data.cpu() # пеневодим в float, потому что от int нельзя взять mean
#     test_accuracy_history.append(accuracy)
    
    print("я еще жив")

    # чтобы избежать утечки памяти, нужно обязательно поставить .data, иначе объект, 
    # который мы кладем в list, будет хранить в себе весь граф вычислений
    # в loss хранится весь граф, который нам помогает потом градиенты обсчитать.

In [ ]:
lenet5.forward(X_test)

In [ ]:
# plt.plot(test_accuracy_history);
plt.plot(test_loss_history);

In [ ]:
# для подбора нужного паддинга

import torch

N = 4
C = 3
C_out = 10
H = 8
W = 16

x = torch.ones((N, C, H, W))

# torch.Size([4, 10, 9, 17])
out8 = torch.nn.Conv2d(C, C_out, kernel_size=(2, 2), padding=1)(x)
print(out8.shape) # для самопроверки